### Create a new environment to keep things isolated
```
pyenv virtualenv sepspark
pyenv activate sepspark
```
### Install the required modules
`pip install pyspark trino pystarburst`

In [57]:
# Lets try to access the Trino Query using the spark
import os
import trino
from pyspark.sql import SparkSession
from pystarburst import Session

# Create a SparkSession
spark = SparkSession.builder \
    .appName("SEP Sparked") \
    .master("local") \
    .config("spark.jars", "./trino-jdbc-435.jar") \
    .config("spark.driver.extraClassPath", "./trino-jdbc-435.jar") \
    .config("spark.executor.extraClassPath", "./trino-jdbc-435.jar") \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

# Define the JDBC connection properties
url = "jdbc:trino://XXXURLXXXX"

# Read data from the database using Spark JDBC
df = spark.read \
    .format("jdbc") \
    .option("driver","io.trino.jdbc.TrinoDriver") \
    .option("url", url) \
    .option("dbtable", "customer") \
    .option("roles","system:sysadmin") \
    .option("user","XXUSERXXX") \
    .load()

# Show the dataframe
df.show()

# Lets use the PyStarburst to connect to the Trino
# Create a PyStarburst session
# Connect to SEP
SEPConnStr = {
    "host": "http://XXXXURLXXXX",
    "port": 8080,
    "user": "XXXXXUSERXXXXXX",
    "roles": {"system": "sysadmin"},
}
session = Session.builder.configs(SEPConnStr).create()
psdf = session.table("tpch.tiny.customer").show()

# Lets create an empty SparkDataframe and fetch the results to the SparkDataframe
spark_df = spark.createDataFrame([], df.schema)
spark_df = session.table("tpch.tiny.customer").limit(20)
spark_df.show()

# Stop the SparkSession
spark.stop()


------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"custkey"  |"name"              |"address"                              |"nationkey"  |"phone"          |"acctbal"  |"mktsegment"  |"comment"                                           |
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|1          |Customer#000000001  |IVhzIApeRb ot,c,E                      |15           |25-989-741-2988  |711.56     |BUILDING      |to the even, regular platelets. regular, ironic...  |
|2          |Customer#000000002  |XSTf4,NCwDVaWNe6tEgvwfmRchLXak         |13           |23-768-687-3665  |121.65     |AUTOMOBILE    |l accounts. blithely ironic theodolites integra...  |
|3          |Customer#000000003  |MG9kdTD2WBHm                   